In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_item_number = df['Item_Number'].tolist()

print('总数量：' + str(len(list_item_number)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import random
import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for item_number in list_item_number:
    work.put_nowait(item_number)

def crawler():
    global df
    
    while not work.empty():
        item_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                list_appid = ['tfy6d12dc-19a5-4093-9f00-f276d950f77',
                              'picheng11-5163-4258-be21-a5629527a07',
                              'Shanghai-b2aa-4eed-82c3-b1e42e2e9179',
                              'danjidin-f8b1-4e84-adce-026944025ceb',
                              'ibay365a-b93e-46e7-8b46-6a33b716ab2a',
                              '-IBAY365-PRD-08e35c535-85b731b3']

                appid_index = random.randint(0, 5)

                url_developer = 'http://106.75.138.2/index.php/api/GetAppToken?auth=757F054AA22E1711B598A89EB276EDC193933EC8&appid=' + list_appid[appid_index]
                resp = requests.get(url_developer,
                                    proxies=Proxy.get_Proxy_Requests()).text
                token = json.loads(resp)['getapptokenResponse']['token']

                # = = = = = = = = = = = = = = =

                ua = UA.get_User_Agent_Requests()
                ua['X-EBAY-API-IAF-TOKEN'] = token
                resp = requests.get('https://open.api.ebay.com/shopping?callname=GetSingleItem&appid=' + list_appid[appid_index] + '&responseencoding=JSON&siteid=0&version=967&IncludeSelector=Details,Compatibility,ItemSpecifics&ItemID=' + item_number,
                                    headers=ua,
                                    proxies=Proxy.get_Proxy_Requests()).text

                info = json.loads(resp)
                if info['Ack'] != 'Success':
                    continue

                # = = = = = = = = = = = = = = =

                title = info['Item']['Title'].strip()

                # = = = = = = = = = = = = = = =

                price = info['Item']['CurrentPrice']['CurrencyID'].strip() + ' ' + str(info['Item']['CurrentPrice']['Value'])

                # = = = = = = = = = = = = = = =

                sold = info['Item']['QuantitySold']

                # = = = = = = = = = = = = = = =

                if 'ItemCompatibilityList' in info['Item']:
                    vehicle = info['Item']['ItemCompatibilityList']['Compatibility']
                else:
                    vehicle = ''

                # = = = = = = = = = = = = = = =

                dict_vehicle = {}
                for list_name_value in vehicle:
                    make = ''
                    model = ''
                    year = ''
                    for name_value in list_name_value['NameValueList']:
                        if 'Name' in name_value and 'Make' in name_value['Name']:
                            make = name_value['Value'][0].strip()
                        elif 'Name' in name_value and 'Model' in name_value['Name']:
                            model = name_value['Value'][0].strip()
                        elif 'Name' in name_value and 'Year' in name_value['Name']:
                            year = int(name_value['Value'][0].strip())

                    if make not in dict_vehicle:
                        dict_vehicle[make] = {model: [year]}
                    else:
                        if model not in dict_vehicle[make]:
                            dict_vehicle[make][model] = [year]
                        else:
                            dict_vehicle[make][model].append(year)

                # = = = = = = = = = = = = = = =

                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        year_max = max(dict_vehicle[make][model])
                        year_min = min(dict_vehicle[make][model])
                        if year_max == year_min:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                vehicle = '\n'.join(list_vehicle)

                # = = = = = = = = = = = = = = =

                if 'GalleryURL' in info['Item']:
                    src = info['Item']['GalleryURL']
                elif 'PictureURL' in info['Item']:
                    if len(info['Item']['PictureURL']) == 0:
                        src = ''
                    else:
                        src = info['Item']['PictureURL'][0]

                # = = = = = = = = = = = = = = =

                if 'ItemSpecifics' in info['Item']:
                    specification = info['Item']['ItemSpecifics']['NameValueList']
                else:
                    specification = ''

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Item_Number': item_number,
                                         'Title': title,
                                         'Price': price,
                                         'Sold': sold,
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': 'https://www.ebay.com/itm/' + item_number,
                                         'Src': src}])

                for name_value in specification:
                    df_temp[name_value['Name'].strip().lower()] = ';'.join(name_value['Value']).strip()

                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Item_Number': item_number}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(item_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./item_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：706

285312591600  <->  [ok] - 剩余数量：701
364610437874  <->  [ok] - 剩余数量：700
275412606713  <->  [ok] - 剩余数量：699
284385062639  <->  [ok] - 剩余数量：698
235153538096  <->  [ok] - 剩余数量：697
386381025629  <->  [ok] - 剩余数量：696
193313123520  <->  [ok] - 剩余数量：695
165790161882  <->  [ok] - 剩余数量：694
385419996655  <->  [ok] - 剩余数量：693


KeyboardInterrupt
2023-12-13T01:35:36Z


KeyboardInterrupt: 

332363098695  <->  [ok] - 剩余数量：692
